# VISUALIZE_EMERGENT
This file is used to visual KL divergence of distributions of emergent characteristics as well as absolute emergent behvaior values. 

The first step is to load the expert trajectories and compute the baseline values.

Then we load the trajectories for the models that we want to examine.

This involves some reshaping of the trajectories, and unnormalizing. 

Finally, we can compute the values we want, and display them.

In [ ]:
%matplotlib inline

import collections
import matplotlib.pyplot as plt
import numpy as np 
import os
from scipy.stats import gaussian_kde
from scipy.stats import entropy

import hgail.misc.utils

import utils
import visualize_utils

from copy import deepcopy

In [ ]:
def unnormalize(x, mean, std):
    return (x * std) + mean

In [ ]:
def filename2label(fn):
    s = fn.find('-') + 1
    e = fn.rfind('.')
    return fn[s:e]

# Expert stuff

In [ ]:
expert_filepath = '../../data/trajectories/ngsim_all.h5'

filenames = list(utils.NGSIM_FILENAME_TO_ID.keys())

expert = dict()
for fn in filenames:
    t = filename2label(fn)
    print("fn:", fn, "t:", t)
    expert[t] = utils.load_data(
        expert_filepath, 
        min_length=250, 
        normalize_data=False, 
        clip_std_multiple=10.,
        ngsim_filename=fn
    )

In [ ]:
x, names = utils.load_x_feature_names(expert_filepath, 'trajdata_i101_trajectories-0750am-0805am.txt')
observation_indexes = dict([(names[i], i) for i in range(len(names))])
#print(observation_indexes)

In [ ]:
print(names)

In [ ]:
# modifies expert_values and bounds in place
def compute_expert_for_timeperiod(obser_attrs, key_for_attr, expert_values, expert_bounds, expert,
                                  time_period='0750am-0805am'):
    expert_values[time_period] = dict()
    expert_bounds[time_period] = dict()
    t = time_period # so i dont have to write it out every time
    for attr in obser_attrs:
        expert_values[time_period][key_for_attr[attr]] = gaussian_kde(expert[time_period]\
                                                                      ['observations'][:,observation_indexes[attr]])
        #lower = np.percentile(expert[time_period]['observations'][:,observation_indexes[attr]], 1)
        #upper = np.percentile(expert[time_period]['observations'][:,observation_indexes[attr]], 99)
        lower = np.min(expert[time_period]['observations'][:,observation_indexes[attr]])
        upper = np.max(expert[time_period]['observations'][:,observation_indexes[attr]])
        expert_bounds[time_period][key_for_attr[attr]] = (lower, upper)

    # Some specific formatting ones
    expert_values[time_period]['iTTC'] = gaussian_kde(1/expert[time_period]\
                                                      ['observations'][:,observation_indexes['time_to_collision']])
    #lower = np.percentile(1/expert[time_period]['observations'][:,observation_indexes['time_to_collision']], 1)
    #upper = np.percentile(1/expert[time_period]['observations'][:,observation_indexes['time_to_collision']], 99)
    lower = np.min(1/expert[time_period]['observations'][:,observation_indexes['time_to_collision']])
    upper = np.max(1/expert[time_period]['observations'][:,observation_indexes['time_to_collision']])
    expert_bounds[time_period]['iTTC'] = (lower, upper)


In [ ]:
observation_attributes = ['accel','turn_rate_global',
                          'velocity', 'jerk', 'time_to_collision', 
                          'distance_road_edge_right', 'distance_road_edge_left']
key_for_attr = {'velocity': 'Speed',
                'jerk': 'Jerk',
                'time_to_collision': 'TTC',
                'distance_road_edge_left': 'dleft',
                'distance_road_edge_right': 'dright',
                'accel': 'Acceleration',
                'turn_rate_global': 'Turn-rate'}

expert_values = dict()
expert_bounds = dict()
timeperiod_labels = [filename2label(fn) for fn in filenames]


for time_period in timeperiod_labels:
    compute_expert_for_timeperiod(observation_attributes, key_for_attr, expert_values, expert_bounds, 
                                  expert, time_period)
    

In [ ]:
print(expert_bounds)

# Load model stuff

In [ ]:
model_labels = [
    'singleagent_def_1_fine',
    'singleagent_def_2_fine',
    'singleagent_def_3_fine',
    'multiagent_curr_1_fine',
    'multiagent_curr_2_fine',
    'multiagent_curr_3_fine'
]
n_itrs = dict([(i, 200) for i in model_labels])
for i in model_labels:
    if 'multi' in i:
        n_itrs[i] = 200



In [ ]:
orig_traj_lab_dict = visualize_utils.get_trajs_dict(model_labels)
valdirs, params_filepaths = visualize_utils.get_val_dirs_and_params_paths_d(model_labels, n_itrs)

## traj_lab_dict Structure:

* for each model, a tuple of (trajectories, labels) that are returned from utils.load_trajs_labels()
    
    * trajectories is a list 
    
        * there is a list element for each dataset (e.g. i101_750_805, i101_805_815..., so 6 elements)
        
            * This is determined from the validate run. Each dataset that we generate trajectories for (filenames variable in the main function for validate.py)
        
        * each list element is an ndarry of shape (T,), where T = number of simulations run. 
                * This is determined in validate.py, as run_args.n_multiagent_trajs / args.n_envs. For n_trajs = 10000, n_vehs = 100, T = 100.
        
                * validate.py produces a total of run_args.n_multiagent_trajs trajectories. From each initial sample scene, we simulate 1 trajectory for each agent. Thus, n_vehs trajectories. Therefore, we need to run T simulations. 
         
            * each item in the array is a dictionary, of trajectory 'characteristics'
            
               * a trajectory component is something like, observations, actions:
           
                   * dict_keys(['rewards', 'rmse_t', 'prev_action', 'x', 'observations', 'mean', 'is_colliding', 'actions', 'y', 'log_std', 'rmse_vel', 'rmse_pos', 'phi', 's'])
                  
                   * the values are an array of shape (H, N, K), where H = the env_H (length of a trajectory or number of timesteps). N = number of agents. K is a third dimension, for things like observations (for which K=64). For something like rmse_t, K doesn't exist (array of shape (H, N)).
                   
                   * There is essentially a dictionary for every intial scene that we sampled. 
    
    - labels is a list
    
        * There is also an element for each dataset, like for trajectories. So, 6 in the default case we have.
    
            + ['0750am-0805am', '0805am-0820am', '0820am-0835am', '0400-0415', '0500-0515', '0515-0530']
    
        

In [ ]:
# This is where we unnormalize / reshape for the multiagent viz case
# reshape basically flattens each attribute of the trajectory
def reformat_trajectories(traj_lab_dict, length = 200, multi=True, reshape=True, 
                          unnormalize_data=True):
    for i, model in enumerate(model_labels):
        trajs = traj_lab_dict[model][0]
        params = hgail.misc.utils.load_params(params_filepaths[i])
        for timeperiod in trajs:
            for traj in timeperiod:
                if multi:
                    n_veh = traj['observations'].shape[1]
                else:
                    n_veh = 1
                if unnormalize_data:
                    for j in range(n_veh):
                        traj['observations'][:,j] = unnormalize(
                            traj['observations'][:,j], 
                            params['normalzing']['obs_mean'],
                            np.sqrt(params['normalzing']['obs_var'])
                        )
                if reshape:
                    for attr in traj.keys():
                        shape = traj[attr].shape
                        if multi:
                            if len(shape) > 0 and shape[0] == length and shape[1] == n_veh:
                                traj[attr] = traj[attr].reshape(length*n_veh, -1)
                            else:
                                print(attr)
                                print(traj[attr].shape)


In [ ]:
traj_lab_dict = orig_traj_lab_dict#deepcopy(orig_traj_lab_dict) #sometimes crashes kernel to deepcopy...
reformat_trajectories(traj_lab_dict, reshape=True, unnormalize_data=True, length=200)


# Define functions that will be used for analysis

In [ ]:
def compute_kl(ref, comp, x, eps=1e-3):
    return entropy(ref.pdf(x) + eps, comp.pdf(x) + eps)
    logp = ref.logpdf(x)
    p = np.exp(logp)
    logq = comp.logpdf(x)
    return - np.sum(p * (logq - logp))

In [ ]:
def compute_emergent_divergence(trajs, labels, bounds, expert, nbins=250):
    # collect values
    emergent = dict()
    
    for (timeperiod, l) in zip(trajs, labels):
        emergent[l] = dict()
        emergent[l]['Acceleration'] = np.concatenate([traj['observations'][:,observation_indexes['accel']] for traj in timeperiod])
        emergent[l]['Turn-rate'] = np.concatenate([traj['observations'][:,observation_indexes['turn_rate_global']] for traj in timeperiod])
        emergent[l]['Speed'] = np.concatenate([traj['observations'][:,observation_indexes['velocity']] for traj in timeperiod])
        emergent[l]['Jerk'] = np.concatenate([traj['observations'][:,observation_indexes['jerk']] for traj in timeperiod])
        #emergent[l]['TTC'] = np.concatenate([traj['observations'][:,observation_indexes['time_to_collision']] for traj in timeperiod])
        emergent[l]['iTTC'] = np.concatenate([1/traj['observations'][:,observation_indexes['time_to_collision']] for traj in timeperiod])        
        
    # compute divergences
    divergences = dict()
    for l, l_values in emergent.items():
        
        if l not in expert.keys():
            continue
            
        divergences[l] = dict()
        for k, values in l_values.items():
            print(k, l)
            x = np.linspace(bounds[l][k][0], bounds[l][k][1], nbins)
            divergences[l][k] = compute_kl(expert[l][k], gaussian_kde(values), x)
            
    return divergences

In [ ]:
def compute_divs(traj_lab_dict, model_labels, expert_bounds, expert_values):
    model_divs = []
    ngsim_labels = traj_lab_dict[model_labels[0]][1]
    model_trajs = [traj_lab_dict[key][0] for key in model_labels]
    for i, trajs in enumerate(model_trajs):
        print('computing divergence for model {}'.format(model_labels[i]))
        divs = compute_emergent_divergence(trajs, ngsim_labels, expert_bounds, expert_values)
        model_divs.append(divs)
    return model_divs

In [ ]:
def do_plots(model_divs, name='emergent'):
    plt.figure(figsize=(20,4))
    attr_width = .2
    for i, divs in enumerate(model_divs):

        for (timeperiod, timeperiod_divs) in divs.items():
            attr_keys = sorted(timeperiod_divs.keys())
            for j, attr in enumerate(attr_keys):
                plt.subplot(1,len(attr_keys),j+1)
                plt.title(attr)
                div = timeperiod_divs[attr]
                plt.bar(i*attr_width, div, width=.15, label=model_labels[i])
                if i == len(model_divs) - 1:
                    plt.legend()
    plt.savefig(name+'.png')
    plt.tight_layout()

# Run the below 2 cells after redefining the models

In [ ]:
model_divs = compute_divs(traj_lab_dict, model_labels, expert_bounds, expert_values)


In [ ]:
do_plots(model_divs)

In [ ]:
def do_plots_2(emergent_values, sorted_order, name='emergent2'):
    plt.figure(figsize=(20,4))
    attr_width = .2
    for i, model in enumerate(sorted_order):
        divs = emergent_values[model]
        attr_keys = sorted(emergent_values[model].keys())
        for j, attr in enumerate(attr_keys):
            plt.subplot(1,len(attr_keys),j+1)
            plt.title(attr)
            div = divs[attr]
            plt.bar(i*attr_width, div, width=.15, label=model)
            if i == len(emergent_values) - 1:
                plt.legend()
    plt.savefig(name+'.png')
    plt.tight_layout()

In [ ]:

ngsim_labels = traj_lab_dict[model_labels[0]][1]

for t in ngsim_labels:
    print(t)
    model_divs_avg = dict()
    model_divs_avg['average_single'] = dict()
    model_divs_avg['average_multi'] = dict()
    for attr in model_divs[0][t].keys():
        model_divs_avg['average_single'][attr] = np.mean([model_divs[i][t][attr] \
                                                            for i, label in enumerate(model_labels) \
                                                              if 'single' in label])
        model_divs_avg['average_multi'][attr] = np.mean([model_divs[i][t][attr]\
                                                            for i, label in enumerate(model_labels) \
                                                              if 'multi' in label])
    avg_sorted_order = ['average_single', 'average_multi']
    do_plots_2(model_divs_avg, avg_sorted_order, 'emergent_avg_'+t)
    

In [ ]:

ngsim_labels = traj_lab_dict[model_labels[0]][1]

to_average = ['0805am-0820am', '0820am-0835am']
n = len(to_average)


model_divs_avg = dict()
model_divs_avg['average_single'] = dict()
model_divs_avg['average_multi'] = dict()

for t in to_average:
    print(t)
    for attr in model_divs[0][t].keys():
        if attr in model_divs_avg['average_single'].keys():
            model_divs_avg['average_single'][attr] += np.mean([model_divs[i][t][attr] \
                                                                for i, label in enumerate(model_labels) \
                                                                  if 'single' in label])/n
            model_divs_avg['average_multi'][attr] += np.mean([model_divs[i][t][attr]\
                                                                for i, label in enumerate(model_labels) \
                                                                  if 'multi' in label])/n
        else:
            model_divs_avg['average_single'][attr] = np.mean([model_divs[i][t][attr] \
                                                                for i, label in enumerate(model_labels) \
                                                                  if 'single' in label])/n
            model_divs_avg['average_multi'][attr] = np.mean([model_divs[i][t][attr]\
                                                                for i, label in enumerate(model_labels) \
                                                                  if 'multi' in label])/n
avg_sorted_order = ['average_single', 'average_multi']
print(model_divs_avg)
do_plots_2(model_divs_avg, avg_sorted_order, 'emergent_avg_'+'_'.join(to_average))
    

In [ ]:

ngsim_labels = traj_lab_dict[model_labels[0]][1]

to_average = ['0400-0415', '0500-0515', '0515-0530']
n = len(to_average)

model_divs_avg = dict()
model_divs_avg['average_single'] = dict()
model_divs_avg['average_multi'] = dict()

for t in to_average:
    print(t)
    for attr in model_divs[0][t].keys():
        if attr in model_divs_avg['average_single'].keys():
            model_divs_avg['average_single'][attr] += np.mean([model_divs[i][t][attr] \
                                                                for i, label in enumerate(model_labels) \
                                                                  if 'single' in label])/n
            model_divs_avg['average_multi'][attr] += np.mean([model_divs[i][t][attr]\
                                                                for i, label in enumerate(model_labels) \
                                                                  if 'multi' in label])/n
        else:
            model_divs_avg['average_single'][attr] = np.mean([model_divs[i][t][attr] \
                                                                for i, label in enumerate(model_labels) \
                                                                  if 'single' in label])/n
            model_divs_avg['average_multi'][attr] = np.mean([model_divs[i][t][attr]\
                                                                for i, label in enumerate(model_labels) \
                                                                  if 'multi' in label])/n
avg_sorted_order = ['average_single', 'average_multi']
do_plots_2(model_divs_avg, avg_sorted_order, 'emergent_avg_'+'_'.join(to_average))
    

# We also want to compute actual emergent values
These are: Lane change rate, Offroad duration, Collision Rate, and Hard Brake Rate

The above graph is for KL divergence of features, which is not the same as the emergent characteristics of the dataset.

In [ ]:
def calc_offroad(traj, thresh = -1.0):
    num_offroad = float(len(traj['observations'][:,0][
        np.where(np.minimum(traj['observations'][:,observation_indexes['distance_road_edge_left']], 
                            traj['observations'][:,observation_indexes['distance_road_edge_right']]) <= thresh)
        ]))
    return num_offroad / float(len(traj['observations']))

def calc_hard_brake(traj, thresh = -3.0):
    #traj should be unnormalized
    return float(len(traj['observations'][:,observation_indexes['accel']][np.where(\
                traj['observations'][:,observation_indexes['accel']] <= thresh)])) \
                / float(len(traj['actions']))

def calc_collisions(traj, expert=True, H=200, nveh=100):
    if expert:
        return float(len(traj['observations'][:,observation_indexes['is_colliding']][
            np.where(np.isclose(traj['observations'][:,observation_indexes['is_colliding']],1))
        ]))/ float(len(traj['observations']))
    else:
        a = np.reshape(traj['observations'][:,observation_indexes['is_colliding']], (H, nveh))[:50,:]
        return np.mean(np.any(np.isclose(a, 1.0), axis=1))



In [ ]:
def count_num_lane_change(single_traj_d_right, lane_width=3.0, max_jump=1.5, verbose=False):
    num = 0.0
    i = 0
    d0 = single_traj_d_right[0]
    while i < len(single_traj_d_right):
        indxs = np.argwhere(abs(single_traj_d_right[i:] - d0) >= lane_width)
        if verbose:
            print("indxs", indxs)
            print(i)
            print("old d0", d0)
        if len(indxs) == 0: 
            return num
        i += indxs[0][0]
        d0 = single_traj_d_right[i]
        if verbose:
            print(i)
            print(d0)
        if i > 0 and abs(d0 - single_traj_d_right[i-1]) <= max_jump:
            if verbose: print("increase")
            num += 1
    return num

def test_count_num_lane_change():
    single_traj = np.array([0,1.5])
    assert count_num_lane_change(single_traj, lane_width=1.5, max_jump=1, verbose=False) == 0
    single_traj = np.array([0,1,1.5])
    assert count_num_lane_change(single_traj, lane_width=1.5, max_jump=1, verbose=False) == 1

    single_traj = np.array([0,0,0.5,1.1,1,1,1.5,\
                   2.0,2.6,2.4,3.0,\
                  1,2,1.5,2,2.5])
    assert count_num_lane_change(single_traj, lane_width=1.5, max_jump=1) == 3
    single_traj[-1] = 2.4
    assert count_num_lane_change(single_traj, lane_width=1.5, max_jump=1) == 2
    single_traj[-1] = 2.6
    assert count_num_lane_change(single_traj, lane_width=1.5, max_jump=1) == 3

    single_traj = np.array([0,0,5,5,5,5.5,6.5,\
                   2.5,2.6,2.4,3.0,3.6,4.0,\
                  1,2,1.5,2,2.5])
    assert count_num_lane_change(single_traj, lane_width=1.5, max_jump=1, verbose=False) == 3
    
test_count_num_lane_change()

def calc_lane_change(traj):
    num_lane_change = count_num_lane_change(traj['observations'][:,observation_indexes['distance_road_edge_left']])
    return num_lane_change / float(len(traj['observations']))

In [ ]:
# Some other specific metrics we want
HARD_BRAKE_THRESH = -3.0
expert_emergent_values = dict()
for time_period in expert.keys():
    print(time_period)
    expert_emergent_values[time_period] = dict()
    expert_emergent_values[time_period]['Collision Rate'] = calc_collisions(expert[time_period])
    expert_emergent_values[time_period]['Offroad Duration'] = calc_offroad(expert[time_period])
    expert_emergent_values[time_period]['Hard Brake Rate'] = calc_hard_brake(expert[time_period],
                                                                                 thresh = HARD_BRAKE_THRESH)
    #expert_emergent_values[time_period]['Lane Change Rate'] = calc_lane_change(expert[time_period])
    #print(expert_emergent_values[time_period]['Lane Change Rate'])


# Run the below 3 cells after redefining the models

### Small disclaimer that lane change rate is calculated very slowly... 
you may want to run without to start, it isn't super useful. Then start it up and go watch a movie, and maybe it'll be done when you come back :)

In [ ]:
ngsim_labels = traj_lab_dict[model_labels[0]][1]
N_VEH = 100
sorted_order = ['expert']
sorted_order.extend([i for i in sorted(model_labels) if 'single' in i])
sorted_order.extend([i for i in sorted(model_labels) if 'multi' in i])
print(sorted_order)

emergent_values = dict()

for t, t_label in enumerate(ngsim_labels):
    print(t_label)
    emergent_values[t_label] = dict()
    emergent_values[t_label]['expert'] = expert_emergent_values[t_label]
    for label in model_labels:
        print(label)
        trajs = traj_lab_dict[label][0][t]
        emergent_values[t_label][label] = dict()
        emergent_values[t_label][label]['Hard Brake Rate'] = np.mean([calc_hard_brake(scene, thresh = HARD_BRAKE_THRESH
                                                                           ) for scene in trajs])
        emergent_values[t_label][label]['Offroad Duration'] = np.mean([calc_offroad(scene) for scene in trajs])
        emergent_values[t_label][label]['Collision Rate'] = np.mean([calc_collisions(scene, expert=True, nveh=N_VEH) for scene in trajs])
        #emergent_values[t_label][label]['Lane Change Rate'] = np.mean([calc_lane_change(scene) for scene in trajs])

    #do_plots_2(emergent_values[t_label], sorted_order, 'emergent_2_'+t_label)

In [ ]:
for t in ngsim_labels:
    do_plots_2(emergent_values[t], sorted_order, 'emergent_2_'+t)

In [ ]:
avg_sorted_order = ['expert', 'average_single', 'average_multi']
average_emergent_values = dict()
for t, t_label in enumerate(ngsim_labels):
    print(t_label)
    average_emergent_values[t_label] = dict()
    average_emergent_values[t_label]['expert'] = expert_emergent_values[t_label]
    average_emergent_values[t_label]['average_single'] = dict()
    average_emergent_values[t_label]['average_multi'] = dict()

    for attr in average_emergent_values[t_label]['expert'].keys():
        average_emergent_values[t_label]['average_single'][attr] = np.mean([emergent_values[t_label][label][attr] \
                                                                   for label in model_labels if 'single' in label])
        average_emergent_values[t_label]['average_multi'][attr] = np.mean([emergent_values[t_label][label][attr]\
                                                                 for label in model_labels if 'multi' in label])
    do_plots_2(average_emergent_values[t_label], avg_sorted_order, 'emergent_avg2_'+t_label)

In [ ]:
to_average = ['0400-0415', '0500-0515', '0515-0530']
n = len(to_average)

label = "_".join(to_average)
val_average_emergent_values = dict()
val_average_emergent_values[label] = dict()

for model in avg_sorted_order:
    val_average_emergent_values[label][model] = dict()
    for attr in average_emergent_values[to_average[0]][model].keys():
        val_average_emergent_values[label][model][attr] = np.mean([average_emergent_values[t][model][attr] \
                                                               for t in to_average])

do_plots_2(val_average_emergent_values[label], avg_sorted_order, 'emergent_avg2_'+label)

In [ ]:
to_average = ['0805am-0820am', '0820am-0835am']

n = len(to_average)

label = "_".join(to_average)

val_average_emergent_values[label] = dict()

for model in avg_sorted_order:
    val_average_emergent_values[label][model] = dict()
    for attr in average_emergent_values[to_average[0]][model].keys():
        val_average_emergent_values[label][model][attr] = np.mean([average_emergent_values[t][model][attr] \
                                                               for t in to_average])

print(val_average_emergent_values[label])
do_plots_2(val_average_emergent_values[label], avg_sorted_order, 'emergent_avg2_'+label)